In [39]:
from bs4 import BeautifulSoup
import urllib3.request
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium
import os
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

Data downloaded!


In [43]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup


page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
proxy_url = ""

if proxy_url.strip() != "":
    # using proxy
    http = urllib3.ProxyManager(proxy_url)
else:
    # direct internet connection
    http = urllib3.PoolManager()

req = http.request('GET', page_url)
soup = BeautifulSoup(req.data, 'html.parser')

# locate postcode table
toronto_table = soup.find('table',{'class':'wikitable sortable'})

# process table rows and build raw_df
raw_df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
rows = toronto_table.findAll('tr')
for row in rows:
    row_items = row.findAll('td')
    if len(row_items) > 0:
        postcode = row_items[0].text.strip()
        borough = row_items[1].text.strip()
        if borough.lower() != "not assigned":
            neighborhood = row_items[2].text.strip()
            raw_df = raw_df.append({'PostalCode':postcode, 
                                    'Borough':borough, 
                                    'Neighborhood':neighborhood}, 
                                   ignore_index = True)

raw_df.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [44]:
grouped = []
for name, group in raw_df.groupby(['PostalCode', 'Borough'])['Neighborhood']:
    nblist = ''.join(str(x) + ", " for x in group.tolist()).strip(", ")
    if nblist == "Not assigned":
        nblist = name[1]
    grouped.append((name[0], name[1], nblist))

toronto_df = pd.DataFrame(grouped, columns=['PostalCode', 'Borough', 'Neighborhood'])
print(toronto_df.shape)
toronto_df.head()


(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [45]:
toronto_df.tail()

,PostalCode,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


In [46]:
toronto_df.query("Neighborhood == 'Not assigned'")

,PostalCode,Borough,Neighborhood


In [47]:
toronto_df.query("PostalCode == 'M7A'")


,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [51]:

toronto_df.shape

(103, 3)

Part 2: Geocoding


In [56]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

ModuleNotFoundError: No module named 'geocoder'

In [57]:
import csv

with open('Geospatial_Coordinates.csv', 'rt') as geo_file:
    geo_reader = csv.reader(geo_file, delimiter=',')
    for row in geo_reader:
        #print(', '.join(row))
        toronto_df.loc[toronto_df['PostalCode'] == row[0], 'Latitude'] = float(row[1])        
        toronto_df.loc[toronto_df['PostalCode'] == row[0], 'Longitude'] = float(row[2])
        
toronto_df.head()

ValueError: could not convert string to float: 'Latitude'